In [ ]:
##### The function to parse data downloaded from IMDB
##### import the raw data by data_name, and specify the file name of the parsed data frame

def parse_imdb(data_name = "imdb_data.txt", new_name = "imdb_data_parse.txt"):

        import pandas as pd
        import numpy as np
        import re
        import warnings
        warnings.filterwarnings("ignore")

        #### load the imdb data to be parsed
        df = pd.read_csv(data_name)

        ## copy column "mpaa" to store the reason of MPAA rating
        df2 = df
        df2["mpaa_reason"] = df["mpaa"]

        #### for each row of the data
        for i in range(len(df)):

                ### -----------------------------------------------------------------------
                ## for variables with multiple items, return a list (later turn into dummy coding)

                for val in ["genres", "runtimes", "language codes", "languages", "country codes", "countries"]:
                        if type(df.ix[i,val]) == str:  # when the entry is not 'float(nan)'
                                st = df.ix[i,val]
                                value = st[3:-2].split("', u'") 
                                df2[val][i] = value
  
                ### -----------------------------------------------------------------------
                ## extract movie MPAA rating as a string (later turn into factor)

                for val in ["mpaa"]:  
                        if type(df.ix[i,val]) == str:  # when the entry is not 'float(nan)'
                            st = df.ix[i,val]
                            value = re.findall(r'Rated (.*?) for',st)[0] # extract the word between "Rated" and "for"
                            df2[val][i] = value
                ### -----------------------------------------------------------------------
                ## extract the reason of MPAA rating as a string (for text analysis later)

                for val in ["mpaa_reason"]:
                        if type(df.ix[i,val]) == str:  # when the entry is not 'float(nan)'
                            st = df.ix[i,val]
                            value = st.split("for ", 1)[1]
                            df2[val][i] = value
                ### -----------------------------------------------------------------------
                ## for variables in the dictionary format, return a list of:
                ##     - person id 
                ##     - company id (for "production companies" and "distributors")

                for val in ["director", "writer","cast", 
                            "distributors", "producer", "production companies", 
                            "cinematographer", "animation department", 
                                "original music", "editorial department"]:
                        if type(df.ix[i,val]) == str:  # when the entry is not 'float(nan)'
                            st = df.ix[i,val]
                            value = re.findall(r'\d+',st)
                            df2[val][i] = value
                            
                ### -----------------------------------------------------------------------
                ## variables that may not be useful:
                    # ["cover url", "full-size cover url"," canonical title", "canonical title.1", 
                    # "long imdb title", "long imdb canonical title", "smart canonical title", 
                    # "smart long imdb canonical title"]
                    
        df2.to_csv(new_name)
